In [66]:
import duckdb
from polars import DataFrame
from pathlib import Path

In [67]:
Path().absolute().parent.parent

PosixPath('/workspaces/VR-data')

# Alustetaan yhteyttä tietokantaan

In [68]:
project_dir = Path().absolute().parent.parent
db_path = project_dir / "data" / "warehouse" / "vr.duckdb"
print(str(db_path), "  file exists:  ", db_path.is_file())

/workspaces/VR-data/data/warehouse/vr.duckdb   file exists:   True


In [69]:
# funktio kyselyn tekemistä varten ei vaadi yhteyden sulkua koska käytetään WITH
def sql(query: str) -> DataFrame:
    with duckdb.connect(str(db_path), read_only=True) as conn:
        df = conn.execute(query).pl()
    return df


In [86]:
sql("SHOW ALL TABLES")

database,schema,name,column_names,column_types,temporary
str,str,str,list[str],list[str],bool
"""vr""","""medallion""","""my_first_dbt_m…","[""id""]","[""INTEGER""]",false
"""vr""","""medallion""","""my_second_dbt_…","[""id""]","[""INTEGER""]",false
"""vr""","""medallion_bron…","""method_b_train…","[""route_sk"", ""trainNumber"", … ""timeTableRows""]","[""VARCHAR"", ""BIGINT"", … ""STRUCT(stationShortCode VARCHAR, stationUICCode BIGINT, countryCode VARCHAR, ""type"" VARCHAR, trainStopping BOOLEAN, commercialStop BOOLEAN, commercialTrack VARCHAR, cancelled BOOLEAN, scheduledTime VARCHAR, liveEstimateTime VARCHAR, estimateSource VARCHAR, unknownDelay BOOLEAN, actualTime VARCHAR, differenceInMinutes BIGINT, causes STRUCT(passengerTerm VARCHAR, categoryCode VARCHAR, categoryName VARCHAR, validFrom VARCHAR, validTo VARCHAR, id INTEGER, detailedCategoryCode VARCHAR, detailedCategoryName VARCHAR, thirdCategoryCode VARCHAR, thirdCategoryName VARCHAR, description VARCHAR, categoryCodeId INTEGER, detailedCategoryCodeId INTEGER, thirdCategoryCodeId INTEGER)[], trainReady STRUCT(source VARCHAR, accepted BOOLEAN, ""timestamp"" VARCHAR)[])[]""]",false
"""vr""","""medallion_gold…","""method_b_kaj_t…","[""stop_sk"", ""first(stationShortCode)"", … ""lateness_causes_new""]","[""VARCHAR"", ""VARCHAR"", … ""STRUCT(passengerTerm VARCHAR, categoryCode VARCHAR, categoryName VARCHAR, validFrom VARCHAR, validTo VARCHAR, id INTEGER, detailedCategoryCode VARCHAR, detailedCategoryName VARCHAR, thirdCategoryCode VARCHAR, thirdCategoryName VARCHAR, description VARCHAR, categoryCodeId INTEGER, detailedCategoryCodeId INTEGER, thirdCategoryCodeId INTEGER)""]",false
"""vr""","""medallion_silv…","""method_b_silve…","[""stop_sk"", ""first(stationShortCode)"", … ""lateness_causes""]","[""VARCHAR"", ""VARCHAR"", … ""STRUCT(passengerTerm VARCHAR, categoryCode VARCHAR, categoryName VARCHAR, validFrom VARCHAR, validTo VARCHAR, id INTEGER, detailedCategoryCode VARCHAR, detailedCategoryName VARCHAR, thirdCategoryCode VARCHAR, thirdCategoryName VARCHAR, description VARCHAR, categoryCodeId INTEGER, detailedCategoryCodeId INTEGER, thirdCategoryCodeId INTEGER)[]""]",false


In [87]:
sql("SHOW").select("database", "schema", "name")

database,schema,name
str,str,str
"""vr""","""medallion""","""my_first_dbt_m…"
"""vr""","""medallion""","""my_second_dbt_…"
"""vr""","""medallion_bron…","""method_b_train…"
"""vr""","""medallion_gold…","""method_b_kaj_t…"
"""vr""","""medallion_silv…","""method_b_silve…"


# Databasen selailu Silver-tasoa varten

In [72]:
sql("""
WITH exploded AS (
    SELECT 
        route_sk,
        UNNEST(timeTableRows) as timetable_row
    FROM medallion_bronze.method_b_traintest
),

defined AS (
        SELECT 
        route_sk,
        timetable_row::STRUCT(
            stationShortCode STRING,
            stationUICCode INT,
            countryCode STRING,
            type STRING,
            trainStopping BOOL,
            commercialStop BOOL,
            commercialTrack STRING, 
            cancelled BOOLEAN,
            scheduledTime STRING,
            liveEstimateTime STRING,
            estimateSource STRING,
            unknownDelay BOOLEAN, 
            actualTime STRING,
            differenceInMinutes INT,
            causes STRUCT(
                passengerTerm STRING, 
                categoryCode STRING, 
                categoryName STRING, 
                validFrom STRING,
                validTo STRING, 
                id INT, 
                detailedCategoryCode STRING, 
                detailedCategoryName STRING,
                thirdCategoryCode STRING, 
                thirdCategoryName STRING, 
                description STRING,
                categoryCodeId INT, 
                detailedCategoryCodeId INT, 
                thirdCategoryCodeId INT)[],
            trainReady STRUCT(
                source STRING,
                accepted BOOL,
                timestamp STRING)[]
        ) as timetable_struct
    FROM exploded
),
    
flattened AS(
    SELECT 
        md5(route_sk || timetable_struct.stationShortCode || timetable_struct.stationUICCode) as stop_sk,
        route_sk, 
        timetable_struct.*
    FROM defined
)

SELECT    
    stop_sk,    
    FIRST(stationShortCode),    
    FIRST(stationUICCode) as station_id,      
    FIRST(stationShortCode) as station_name,
    MAX(CASE WHEN type = 'ARRIVAL' THEN differenceInMinutes END) AS arrival_lateness, 
    MAX(CASE WHEN type = 'DEPARTURE' THEN differenceInMinutes END) AS departure_lateness,
    FIRST(CASE WHEN type = 'ARRIVAL' THEN scheduledTime END) AS arrival_schedule,    
    FIRST(CASE WHEN type = 'DEPARTURE' THEN scheduledTime END) AS departure_schedule,    
    MAX(CASE WHEN type = 'ARRIVAL' THEN actualTime END) AS arrival_actual,    
    MAX(CASE WHEN type = 'DEPARTURE' THEN actualTime END) AS departure_actual,
    FLATTEN(LIST(causes)) as lateness_causes,

FROM flattened
GROUP BY stop_sk
""")



stop_sk,first(stationShortCode),station_id,station_name,arrival_lateness,departure_lateness,arrival_schedule,departure_schedule,arrival_actual,departure_actual,lateness_causes
str,str,i32,str,i32,i32,str,str,str,str,list[struct[14]]
"""5153b6c1be3948…","""PMK""",551,"""PMK""",1,1,"""2023-11-01T05:…",null,"""2023-11-01T05:…","""2023-11-01T05:…",[]
"""5cd9985ff7680d…","""TNA""",552,"""TNA""",0,0,"""2023-11-01T05:…",null,"""2023-11-01T05:…","""2023-11-01T05:…",[]
"""ffde5b4becf41e…","""KSU""",1128,"""KSU""",1,1,"""2023-11-01T05:…",null,"""2023-11-01T05:…","""2023-11-01T05:…",[]
"""82a7303cc454d6…","""HNN""",1164,"""HNN""",-2,-2,"""2023-11-01T05:…",null,"""2023-11-01T05:…","""2023-11-01T05:…",[]
"""ee84b4e83ebd8a…","""KA""",477,"""KA""",1,1,"""2023-11-01T06:…",null,"""2023-11-01T06:…","""2023-11-01T06:…",[]
"""238eeb6c294023…","""TRÄ""",1290,"""TRÄ""",-2,-2,"""2023-11-01T06:…",null,"""2023-11-01T06:…","""2023-11-01T06:…",[]
"""2bf38673214f2f…","""LRS""",498,"""LRS""",0,0,"""2023-11-01T07:…",null,"""2023-11-01T07:…","""2023-11-01T07:…",[]
"""8a44e9b1272062…","""IMT""",502,"""IMT""",0,0,"""2023-11-01T07:…",null,"""2023-11-01T07:…","""2023-11-01T07:…",[]
"""f47598898d4187…","""SPL""",507,"""SPL""",1,2,"""2023-11-01T08:…",null,"""2023-11-01T08:…","""2023-11-01T08:…",[]


# Databasen selailu Gold-tasoa varten
stop_sk	
first(stationShortCode)	
station_id	
station_name	
arrival_lateness	
departure_lateness	
arrival_schedule	
departure_schedule	
arrival_actual	
departure_actual	
lateness_causes

In [84]:
sql("""
    WITH gold_exploded AS (
    SELECT * EXCLUDE (lateness_causes),
        UNNEST(lateness_causes) as lateness_causes_new
    FROM medallion_silver.method_b_silvertrain
    WHERE station_name LIKE 'KAJ'
)
    
SELECT * FROM gold_exploded;
""")

stop_sk,first(stationShortCode),station_id,station_name,arrival_lateness,departure_lateness,arrival_schedule,departure_schedule,arrival_actual,departure_actual,lateness_causes_new
str,str,i32,str,i32,i32,str,str,str,str,struct[14]
"""68fa3d4531963a…","""KAJ""",387,"""KAJ""",-96,-96,"""2023-11-07T22:…",null,"""2023-11-07T21:…","""2023-11-07T21:…","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}"
"""48dcbc0a6d855b…","""KAJ""",387,"""KAJ""",4,2,"""2023-11-11T07:…",null,"""2023-11-11T07:…","""2023-11-11T07:…","{null,""T"",null,null,null,null,""T1"",null,""T102"",null,null,402720842,18350344,33626113}"
"""b9d0517f02a969…","""KAJ""",387,"""KAJ""",-39,-39,"""2023-11-13T22:…",null,"""2023-11-13T22:…","""2023-11-13T22:…","{null,""E"",null,null,null,null,""E1"",null,null,null,null,335611978,18088200,null}"
"""282822f9117d5b…","""KAJ""",387,"""KAJ""",4,1,"""2023-11-14T12:…",null,"""2023-11-14T12:…","""2023-11-14T12:…","{null,""L"",null,null,null,null,""L2"",null,""L204"",null,null,352389194,34930952,67245313}"
"""ed3f2c2dc5c6ec…","""KAJ""",387,"""KAJ""",-36,-36,"""2023-11-14T22:…",null,"""2023-11-14T22:…","""2023-11-14T22:…","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}"
"""e91492b589ad0d…","""KAJ""",387,"""KAJ""",-29,-29,"""2023-11-15T17:…",null,"""2023-11-15T16:…","""2023-11-15T16:…","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}"
"""9837f1122190ba…","""KAJ""",387,"""KAJ""",8,6,"""2023-11-17T15:…",null,"""2023-11-17T15:…","""2023-11-17T15:…","{null,""M"",null,null,null,null,""M1"",null,null,null,null,453052490,18546952,null}"
"""a6782aabcc40cf…","""KAJ""",387,"""KAJ""",5,null,"""2023-11-17T23:…",null,"""2023-11-17T23:…",null,"{null,""L"",null,null,null,null,""L2"",null,""L201"",null,null,352389194,34930952,16913665}"
"""22d11be81f7d52…","""KAJ""",387,"""KAJ""",5,3,"""2023-11-17T12:…",null,"""2023-11-17T12:…","""2023-11-17T12:…","{null,""M"",null,null,null,null,""M1"",null,null,null,null,453052490,18546952,null}"


In [81]:
sql("""
    WITH gold_exploded AS (
    SELECT * EXCLUDE (lateness_causes),
        UNNEST(lateness_causes) as lateness_causes_new
    FROM medallion_silver.method_b_silvertrain
    WHERE station_name LIKE 'KAJ'
),
defined_causes AS (
    SELECT *,
        lateness_causes_new::STRUCT(
            passengerTerm STRING, 
            categoryCode STRING, 
            categoryName STRING, 
            validFrom STRING,
            validTo STRING, 
            id INT, 
            detailedCategoryCode STRING, 
            detailedCategoryName STRING,
            thirdCategoryCode STRING, 
            thirdCategoryName STRING, 
            description STRING,
            categoryCodeId INT, 
            detailedCategoryCodeId INT, 
            thirdCategoryCodeId INT
        ) as lateness_rows
    FROM gold_exploded
),
gold_lateness AS (
    SELECT *,
        lateness_rows.*
    FROM defined_causes
)
SELECT * FROM gold_lateness;
""")


stop_sk,first(stationShortCode),station_id,station_name,arrival_lateness,departure_lateness,arrival_schedule,departure_schedule,arrival_actual,departure_actual,lateness_causes_new,lateness_rows,passengerTerm,categoryCode,categoryName,validFrom,validTo,id,detailedCategoryCode,detailedCategoryName,thirdCategoryCode,thirdCategoryName,description,categoryCodeId,detailedCategoryCodeId,thirdCategoryCodeId
str,str,i32,str,i32,i32,str,str,str,str,struct[14],struct[14],str,str,str,str,str,i32,str,str,str,str,str,i32,i32,i32
"""68fa3d4531963a…","""KAJ""",387,"""KAJ""",-96,-96,"""2023-11-07T22:…",null,"""2023-11-07T21:…","""2023-11-07T21:…","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}",null,"""E""",null,null,null,null,"""E2""",null,null,null,null,335611978,34865416,null
"""48dcbc0a6d855b…","""KAJ""",387,"""KAJ""",4,2,"""2023-11-11T07:…",null,"""2023-11-11T07:…","""2023-11-11T07:…","{null,""T"",null,null,null,null,""T1"",null,""T102"",null,null,402720842,18350344,33626113}","{null,""T"",null,null,null,null,""T1"",null,""T102"",null,null,402720842,18350344,33626113}",null,"""T""",null,null,null,null,"""T1""",null,"""T102""",null,null,402720842,18350344,33626113
"""b9d0517f02a969…","""KAJ""",387,"""KAJ""",-39,-39,"""2023-11-13T22:…",null,"""2023-11-13T22:…","""2023-11-13T22:…","{null,""E"",null,null,null,null,""E1"",null,null,null,null,335611978,18088200,null}","{null,""E"",null,null,null,null,""E1"",null,null,null,null,335611978,18088200,null}",null,"""E""",null,null,null,null,"""E1""",null,null,null,null,335611978,18088200,null
"""282822f9117d5b…","""KAJ""",387,"""KAJ""",4,1,"""2023-11-14T12:…",null,"""2023-11-14T12:…","""2023-11-14T12:…","{null,""L"",null,null,null,null,""L2"",null,""L204"",null,null,352389194,34930952,67245313}","{null,""L"",null,null,null,null,""L2"",null,""L204"",null,null,352389194,34930952,67245313}",null,"""L""",null,null,null,null,"""L2""",null,"""L204""",null,null,352389194,34930952,67245313
"""ed3f2c2dc5c6ec…","""KAJ""",387,"""KAJ""",-36,-36,"""2023-11-14T22:…",null,"""2023-11-14T22:…","""2023-11-14T22:…","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}",null,"""E""",null,null,null,null,"""E2""",null,null,null,null,335611978,34865416,null
"""e91492b589ad0d…","""KAJ""",387,"""KAJ""",-29,-29,"""2023-11-15T17:…",null,"""2023-11-15T16:…","""2023-11-15T16:…","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}","{null,""E"",null,null,null,null,""E2"",null,null,null,null,335611978,34865416,null}",null,"""E""",null,null,null,null,"""E2""",null,null,null,null,335611978,34865416,null
"""9837f1122190ba…","""KAJ""",387,"""KAJ""",8,6,"""2023-11-17T15:…",null,"""2023-11-17T15:…","""2023-11-17T15:…","{null,""M"",null,null,null,null,""M1"",null,null,null,null,453052490,18546952,null}","{null,""M"",null,null,null,null,""M1"",null,null,null,null,453052490,18546952,null}",null,"""M""",null,null,null,null,"""M1""",null,null,null,null,453052490,18546952,null
"""a6782aabcc40cf…","""KAJ""",387,"""KAJ""",5,null,"""2023-11-17T23:…",null,"""2023-11-17T23:…",null,"{null,""L"",null,null,null,null,""L2"",null,""L201"",null,null,352389194,34930952,16913665}","{null,""L"",null,null,null,null,""L2"",null,""L201"",null,null,352389194,34930952,16913665}",null,"""L""",null,null,null,null,"""L2""",null,"""L201""",null,null,352389194,34930952,16913665
"""22d11be81f7d52…","""KAJ""",387,"""KAJ""",5,3,"""2023-11-17T12:…",null,"""2023-11-17T12:…","""2023-11-17T12:…","{null,""M"",null,null,null,null,""M1"",null,null,null,null,453052490,18546952,null}","{null,""M"",null,null,null,null,""M1"",null,null,null,null,453052490,18546952,null}",null,"""M""",null,null,null,null,"""M1""",null,null,null,null,453052490,18546952,null


In [79]:
sql("""
WITH gold_exploded AS (
    SELECT * EXCLUDE (lateness_causes),
        UNNEST(lateness_causes) as lateness_causes_new

    FROM medallion_silver.method_b_silvertrain
    WHERE station_name LIKE 'KAJ'),

    
defined_causes AS (
    SELECT *,
    lateness_causes_new::STRUCT(
            passengerTerm STRING, 
            categoryCode STRING, 
            categoryName STRING, 
            validFrom STRING,
            validTo STRING, 
            id INT, 
            detailedCategoryCode STRING, 
            detailedCategoryName STRING,
            thirdCategoryCode STRING, 
            thirdCategoryName STRING, 
            description STRING,
            categoryCodeId INT, 
            detailedCategoryCodeId INT, 
            thirdCategoryCodeId INT
    ) as lateness_rows

gold_lateness AS (
    SELECT *,
    lateness_rows.*
    FROM defined_causes
)

SELECT * FROM gold_lateness
""")

ParserException: Parser Error: syntax error at or near "gold_lateness"
LINE 29: gold_lateness AS (
         ^

In [64]:
sql("""
WITH gold_exploded AS (
    SELECT *,
        UNNEST(lateness_causes) as lateness_causes

    FROM medallion_silver.method_b_silvertrain
    WHERE station_name LIKE 'KAJ'),

defined_causes AS (
    SELECT *,
    lateness_causes::Struct(
            passengerTerm STRING, 
            categoryCode STRING, 
            categoryName STRING, 
            validFrom STRING,
            validTo STRING, 
            id INT, 
            detailedCategoryCode STRING, 
            detailedCategoryName STRING,
            thirdCategoryCode STRING, 
            thirdCategoryName STRING, 
            description STRING,
            categoryCodeId INT, 
            detailedCategoryCodeId INT, 
            thirdCategoryCodeId INT
    ) as lateness_rows

gold_lateness AS (
    SELECT *,
    lateness_rows.*
    FROM defined_causes
)

SELECT * FROM gold_lateness
""")



ParserException: Parser Error: syntax error at or near "gold_lateness"
LINE 28: gold_lateness AS (
         ^

In [65]:
sql("""
WITH exploded2 AS (
    SELECT 
        stop_sk,
        UNNEST(lateness_causes) as lateness_causes_unnest
    FROM medallion_silver.method_b_silvertrain
),

defined2 AS (
    SELECT 
        stop_sk,
        lateness_causes_unnest::STRUCT(
            passengerTerm STRING, 
            categoryCode STRING, 
            categoryName STRING, 
            validFrom STRING,
            validTo STRING, 
            id INT, 
            detailedCategoryCode STRING, 
            detailedCategoryName STRING,
            thirdCategoryCode STRING, 
            thirdCategoryName STRING, 
            description STRING,
            categoryCodeId INT, 
            detailedCategoryCodeId INT, 
            thirdCategoryCodeId INT
        ) as lateness_causesss
    FROM exploded2
)

FROM defined2
GROUP BY stop_sk
""")

BinderException: Binder Error: column "lateness_causesss" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(lateness_causesss)" if the exact value of "lateness_causesss" is not important.